In [1]:
import pycram
from pycram.bullet_world import BulletWorld, Object
import pycram.bullet_world_reasoning as btr
import tf
from pycram.designators.motion_designator import MotionDesignatorDescription, MoveArmJointsMotion
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.language import macros, par
from pycram.designators.location_designator import *
from pycram.designators.action_designator import *
from pycram.enums import Arms
from pycram.designators.object_designator import *
from pycram.designators.object_designator import BelieveObject

# %%
import pycram
from pycram.bullet_world import BulletWorld, Object
import pycram.bullet_world_reasoning as btr
import tf
from pycram.designators.motion_designator import MotionDesignatorDescription, MoveArmJointsMotion
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.language import macros, par
from pycram.designators.location_designator import *
from pycram.designators.action_designator import *
from pycram.enums import Arms
from pycram.designators.object_designator import *
from pycram.designators.object_designator import BelieveObject
from pycram.plan_failures import IKError
import math

world = BulletWorld()
world.set_gravity([0, 0, -9.8])
# plane = Object("floor", "environment", "plane.urdf", world=world)
robot = Object("pr2", "robot", "../../resources/" + robot_description.name + ".urdf")
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()

kitchen = Object("kitchen", "environment", "kitchen.urdf")

robot.set_joint_state(robot_description.torso_joint, 0.24)
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

# kitchen.set_color([0.2, 0, 0.4, 0.6])

# %%
spawning_poses = {
    # 'bigknife': Pose([-0.95, 1.2, 1.3], [1, -1, 1, -1]),
    'bigknife': Pose([0.9, 0.6, 1.2], [0, 0, -1, -1]),
    # 'bread': Pose([-0.85, 0.9, 0.90], [0, 0, -1, 1])
    'bread': Pose([-0.85, 0.9, 0.90], [0, 0, 0, 1])
}
bigknife = Object("bigknife", "bigknife", "big-knife.stl", spawning_poses["bigknife"])
bread = Object("bread", "bread", "bread.stl", spawning_poses["bread"])
bigknife_BO = BelieveObject(names=["bigknife"])
bread_BO = BelieveObject(names=["bread"])


def calculate_and_transform_slices(bread_dimension, slice_thickness, bread_pose):
    """
    Calculate number of slices, transform them to the map frame, and provide coordinates with position and orientation.

    Args:
    - bread_dimension (tuple): Dimensions of the bread (length, width, height)
    - slice_thickness (float): Desired thickness of each slice in meters
    - bread_pose (Pose): Spawning pose of the bread with position and orientation

    Returns:
    - num_slices (int): Number of slices
    - coordinates_with_orientation (list): List of slice coordinates with position and orientation
    """

    # Calculate number of slices
    bread_width = bread_dimension[1]
    num_slices = int(bread_width // slice_thickness)

    # Calculate the starting Y-coordinate offset (half the width minus half a slice thickness)
    start_offset = -bread_width / 2 + slice_thickness / 2

    # Calculate slice coordinates
    slice_coordinates = [start_offset + i * slice_thickness for i in range(num_slices)]

    # Transform slice coordinates to map frame with orientation
    coordinates_with_orientation = []
    for coordinate in slice_coordinates:
        x_in_map = bread_pose.position.x
        y_in_map = bread_pose.position.y + coordinate
        z_in_map = bread_pose.position.z

        orientation = [0, 1, 0, 1]
        coordinates_with_orientation.append({
            "position": [x_in_map, y_in_map, z_in_map],
            "orientation": orientation
        })

    return num_slices, coordinates_with_orientation


def axis_angle_to_quaternion(axis, angle):
    """
    Convert axis-angle to quaternion.
    axis: (x, y, z) tuple representing rotation axis.
    angle: rotation angle in radians.
    """
    axis_length = math.sqrt(sum([i ** 2 for i in axis]))
    normalized_axis = tuple(i / axis_length for i in axis)

    w = math.cos(angle / 2)
    x = normalized_axis[0] * math.sin(angle / 2)
    y = normalized_axis[1] * math.sin(angle / 2)
    z = normalized_axis[2] * math.sin(angle / 2)

    return (w, x, y, z)


def multiply_quaternions(q1, q2):
    """
    Multiply two quaternions.
    q1, q2: (w, x, y, z) tuple representing quaternion.
    
    """
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2

    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2

    return (w, x, y, z)


with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.33]).resolve().perform()

    pickup_pose_knife = CostmapLocation(target=bigknife_BO.resolve(), reachable_for=robot_desig).resolve()
    pickup_arm = pickup_pose_knife.reachable_arms[0]

    NavigateAction(target_locations=[pickup_pose_knife.pose]).resolve().perform()

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/oem/pycram_ws/src/pycram/src/pycram/bullet_world.py", line 536, in run
    basePosition=cameraTargetPosition)
UnboundLocalError: local variable 'cameraTargetPosition' referenced before assignment


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 2060/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 525.125.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 525.125.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 2060/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Scalar element defin

In [5]:
with simulated_robot:
    PickUpAction(object_designator_description=bigknife_BO,
                 arms=["left"],
                 grasps=["top"]).resolve().perform()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [5]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.33]).resolve().perform()

    pickup_pose_knife = CostmapLocation(target=bigknife_BO.resolve(), reachable_for=robot_desig).resolve()
    pickup_arm = pickup_pose_knife.reachable_arms[0]

    NavigateAction(target_locations=[pickup_pose_knife.pose]).resolve().perform()


TypeError: super(type, obj): obj must be an instance or subtype of type

In [3]:
world.reset_bullet_world()

In [11]:
with simulated_robot:
      ParkArmsAction([Arms.BOTH]).resolve().perform()
      world.remove_vis_axis()
      print (bigknife.get_pose())
      world.add_vis_axis(bigknife.get_pose())

header: 
  seq: 0
  stamp: 
    secs: 1694253244
    nsecs:  40179252
  frame_id: "map"
pose: 
  position: 
    x: 0.9
    y: 0.6
    z: 1.2
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.7071067811865475
    w: 0.7071067811865476
